In [1]:
from workload_generator import *

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pretrained model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Example usage
input_text = "Hello, how are you today?"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded_output)


/home/jeffjw/anaconda3/envs/DWG/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/jeffjw/anaconda3/envs/DWG/lib/python3.8/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Hello, how are you today?

I'm so happy to be here. I'm so


In [5]:
import torch.onnx

# Set the path for saving the ONNX file
onnx_file_path = "model.onnx"

# Set the input tensor
input_ids = torch.tensor([[15496, 11, 703, 389, 345, 30, 44]])

# Export the model to ONNX
torch.onnx.export(model, input_ids, onnx_file_path)


In [6]:
import onnx
import numpy as np
import onnxruntime as ort
#session = ort.InferenceSession("model.onnx", 
#                               providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
#                               )
#ortvalue = ort.OrtValue.ortvalue_from_numpy(input_ids.numpy())
onnx_model = onnx.load("model.onnx")
#onames = [o.name for o in onnx_model.graph.output]

#session.run(onames, {"onnx::Reshape_0": ortvalue})

In [7]:
onnx_model = onnx.shape_inference.infer_shapes(onnx_model)

In [8]:
# Generate Value Dictionary
value_info = onnx_model.graph.value_info
value_dict = {reformat_name(v.name): v for v in value_info}

# Also include the output of graph to the value_dict
for out in onnx_model.graph.output:
    value_dict[reformat_name(out.name)] = out

In [13]:
for node in onnx_model.graph.node:
    if reformat_name(node.name) == 'transformer.h.0.attn.MatMul':
        break

AttributeError: keys

In [ ]:
from typing import NamedTuple, Optional, Dict

class ParsedModel(NamedTuple):
    """
    Parsed Model: Required: op_type, input_shapes, output_shapes, macs
    Optional: attributes, param_shapes
    op_type: str, just the type of operation
    input_shapes: input shapes of the operation, list of int tuples
    output_shapes: output shapes of the operation, list of int tuples
    macs: number of Multiply-Accumulate operations
    attributes: optional attributes of the operation, dictionary
    param_shapes: optional parameter shapes of the operation, list of int tuples
    """
    op_type: str
    input_shapes: list
    output_shapes: list
    macs: int
    attribute: Optional[list]
    param_shapes: Optional[list] = []

def reformat_name(tensor_name):
    """
    Reformat tensor name if it starts with "/", so that all "/" are changed to "." 
    and the first "." is removed.
    """
    if tensor_name.startswith("/"):
        tensor_name = tensor_name[1:]
    return tensor_name.replace("/", ".")


def parse_convnode(node, value_dict, param_dict):
    """
    Parse Convolutional node to include input and output shapes.
    Count MACs for Convolutional layers.
    """
    # Get the input size
    input_name = reformat_name(node.input[0])
    input_shape = value_dict[input_name].type.tensor_type.shape.dim
    input_channels = input_shape[1].dim_value
    # Get the output size
    output_name = reformat_name(node.output[0])
    output_shape = value_dict[output_name].type.tensor_type.shape.dim
    output_channels = output_shape[1].dim_value
    # Get the kernel size
    kernel_name = reformat_name(node.input[1])
    kernel_shape = param_dict[kernel_name].dims
    kernel_size = kernel_shape[-2:]
    # Also check for bias
    bias_shape = None
    if len(node.input) == 3:
        bias_name = reformat_name(node.input[2])
        bias_shape = param_dict[bias_name].dims
        bias_size = bias_shape[0]
    # Get the number of MACs
    macs = np.prod(kernel_size) * input_channels * output_channels * np.prod(output_shape[2:]) + (0 if bias_shape is None else output_channels * np.prod(output_shape[2:]))
    # Create ParsedModel object.
    # First map input_shape and output_shape to a single element list 
    if bias_shape is not None:
        input_shape = [tuple([d.dim_value for d in input_shape]), tuple([d for d in kernel_shape]), tuple([d for d in bias_shape])]
    else:
        input_shape = [tuple([d.dim_value for d in input_shape]), tuple([d for d in kernel_shape])]
    output_shape = [tuple([d.dim_value for d in output_shape])]
    # Create param_shapes
    param_shapes = [tuple([d for d in kernel_shape])]
    return ParsedModel(node.op_type, input_shape, output_shape, macs, attribute=node.attribute, param_shapes=param_shapes)

def parse_gemmnode(node, value_dict, param_dict):
    """
    Parse Gemm node to include input and output shapes.
    Count MACs for Gemm operations.
    """
    param_shapes = []
    # Get the input size for #0
    i1_name = reformat_name(node.input[0])
    # Check if i1 in param dict
    if i1_name in param_dict:
        param_shapes.append(tuple([d for d in param_dict[i1_name].dims]))
        i1_channels = param_dict[i1_name].dims[0]
        internal_channels = param_dict[i1_name].dims[1]
    else:
        i1_shape = value_dict[i1_name].type.tensor_type.shape.dim
        internal_channels = i1_shape[1].dim_value
        i1_channels = i1_shape[0].dim_value
    # Get the input size for #1
    i2_name = reformat_name(node.input[1])
    # Check if i2 in param dict
    if i2_name in param_dict:
        param_shapes.append(tuple([d for d in param_dict[i2_name].dims]))
        i2_channels = param_dict[i2_name].dims[1]
    else:
        i2_shape = value_dict[i2_name].type.tensor_type.shape.dim
        i2_channels = i2_shape[1].dim_value
    
    # Check if bias is present
    bias_shape = None
    if len(node.input) == 3:
        bias_name = reformat_name(node.input[2])
        bias_shape = param_dict[bias_name].dims
        bias_size = bias_shape[0]
        param_shapes.append(tuple([d for d in bias_shape]))
    # Get the number of MACs
    macs = i1_channels * internal_channels * i2_channels
    if bias_shape is not None:
        input_shapes = [tuple([i1_channels, internal_channels]), tuple([internal_channels, i2_channels]), tuple([i2_channels])]
    else:
        input_shapes = [tuple([i1_channels, internal_channels]), tuple([internal_channels, i2_channels])]
    return ParsedModel(node.op_type, input_shapes, [tuple([i1_channels, i2_channels])], macs, attribute=node.attribute, param_shapes=param_shapes)

def parse_onnx(onnx_model):
    """
    Parse ONNX model file to include input and output shapes.
    Cout MACs for Convolutional layers, Gemm, and Matmul operations.
    """
    # First check if op type is supported
    supported_ops = ["Conv", "MatMul", "Gemm"]
    parsed_model = {}
    # Parsed model should have the following fixed fields in NamedTuple:
    # op_type, input_shapes, output_shapes, macs, attributes (optional), param_shapes (optional, only if in graph.initializer)
    warning_issued=False
    # If onnx_model's value info is empty list, generate with shape inference
    if not onnx_model.graph.value_info:
        onnx_model = onnx.shape_inference.infer_shapes(onnx_model)
    value_info = onnx_model.graph.value_info
    value_dict = {reformat_name(v.name): v for v in value_info}

    # Also include the output of graph.node to the value_dict
    for out in onnx_model.graph.output:
        value_dict[reformat_name(out.name)] = out

    param_info = onnx_model.graph.initializer
    param_dict = {p.name: p for p in param_info}

    for node in onnx_model.graph.node:
        # Check if the node is supported
        if not any([op_type in node.op_type for op_type in supported_ops]):
            if not warning_issued:
                print(f"Warning: {node.op_type} is not supported.")
                warning_issued = True

        # Count MACs for Conv
        if "Conv" in node.op_type:
            parsed_model[reformat_name(node.name)] = parse_convnode(node, value_dict, param_dict)
        # Count MACs for Gemm
        elif "Gemm" in node.op_type:
            parsed_model[reformat_name(node.name)] = parse_gemmnode(node, value_dict, param_dict)
        # Count MACs for MatMul
        elif "MatMul" in node.op_type:
            parsed_model[reformat_name(node.name)] = parse_gemmnode(node, value_dict, param_dict)
        
        # Only Parsing Conv, GEMM, Matmul for now. Add more as needed.
    return parsed_model

In [ ]:
parsed_model = parse_onnx(onnx_model)

In [ ]:
onnx_model

In [ ]:
write_csv(parsed_model, filename='NotUnique7.csv', unique_only=False)

In [ ]:
# Analyze Gemm type.
#  Get all "gemm" nodes
value_info = onnx_model.graph.value_info
value_dict = {reformat_name(v.name): v for v in value_info}
for out in onnx_model.graph.output:
    value_dict[reformat_name(out.name)] = out

param_info = onnx_model.graph.initializer
param_dict = {p.name: p for p in param_info}

gemm_nodes = [node for node in onnx_model.graph.node if "Gemm" in node.op_type]
node = gemm_nodes[0]

In [ ]:
parse_gemmnode(node, value_dict, param_dict)

In [ ]:
i1_name = reformat_name(node.input[0])
i2_name = reformat_name(node.input[1])
# Check if i1 in param dict
if i1_name in param_dict:
    i1_channels = param_dict[i1_name].dims[0]
    internal_channels = param_dict[i1_name].dims[1]
else:
    i1_shape = value_dict[i1_name].type.tensor_type.shape.dim
    internal_channels = i1_shape[1].dim_value
    i1_channels = i1_shape[0].dim_value
if i2_name in param_dict:
    i2_channels = param_dict[i2_name].dims[1]
else:
    i2_shape = value_dict[i2_name].type.tensor_type.shape.dim
    i2_channels = i2_shape[1].dim_value

i1_channels

In [ ]:
if not onnx_model.graph.value_info:
    onnx_model = onnx.shape_inference.infer_shapes(onnx_model)
value_info = onnx_model.graph.value_info
value_dict = {reformat_name(v.name): v for v in value_info}
for out in onnx_model.graph.output:
    value_dict[reformat_name(out.name)] = out

param_info = onnx_model.graph.initializer
param_dict = {p.name: p for p in param_info}

In [ ]:
#  Get all "matmul" or "gemm" nodes
value_dict
matmul_nodes = [node for node in onnx_model.graph.node if "MatMul" in node.op_type]

In [ ]:
value_dict[reformat_name(matmul_nodes[-1].input[0])]

In [ ]:
parsed_model = parse_onnx(onnx_model)
for parse

In [ ]:
onnx_model.graph.output[2]

In [ ]:
tr_dict = {}
for node in onnx_model.graph.node:
    if "Transpose" in node.op_type:
        for i, output in enumerate(node.output):
            if reformat_name(output) not in value_dict:
                print(output)


In [ ]:
for node in onnx_model.graph.node:
    # Check if "value.59" is in the node's inputs or outputs
    if 'value.59' in node.output:
        print(f'Found "value.59" in node: {node.name}')

In [ ]:
value_dict['/transformer/h.0/Add_1_output_0']